In [1]:
from Bio.SeqIO.FastaIO import SimpleFastaParser  # low level fast fasta parser
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
sns.set()

## 1. functions

In [2]:
def get_ref(in_file):
    """deriving the first record 
    from the input fasta
    """
    
    with open(in_file) as in_handle:
        for record in SimpleFastaParser(in_handle):
            # sequence is under the 2nd index in the record's tuple
            ref_seq = record[1] 
            print(record)
            break
    
    return ref_seq
    
    

In [3]:
def get_duplex_posits(ref_seq, ref_nuc):
    """takes duplex positions out of the reference string
    """
    nucleotides = ['A', 'T', 'G', 'C']
    start = 0
    
    d = {'start_pos': [], 'stop_pos': [], 'ref_duplex': [] }
    
    while start != len(ref_seq) - 1:
        
        if ref_seq[start] == ref_nuc:
            first_duplex_nuc = ref_seq[start]
            first_duplex_pos = start
            
            # + 1 to get slice from the next nuc
            second_duplex_pos = start + 1
            for nuc in ref_seq[start + 1 : ]:
                if nuc in nucleotides:
                    second_duplex_nuc = nuc
                    break
                else:
                    second_duplex_pos += 1
                        
            d['start_pos'].append(first_duplex_pos)
            d['stop_pos'].append(second_duplex_pos)
            d['ref_duplex'].append(first_duplex_nuc + second_duplex_nuc)
            # incrementing index to move forward the string
            start += 1
    
        else:
            # incrementing anyway
            start += 1
    
    return d
                
    
                    


In [4]:
def create_df(duplex_posits, snp_type):

    df = pd.DataFrame(duplex_posits)
    
    for snp in snp_type:
        df[snp] = np.nan
    
    df.fillna(inplace=True, value=0)
    
    
    
    return df
    

In [5]:
def collect_snp(in_file, df, nuc_to_watch):
    coverage = 0
    
    with open(in_file) as in_handle:
        
        reads = SimpleFastaParser(in_handle)
        for record in reads:
            coverage += 1
            
            read = record[1]
            for row in df.index:
                if read[df.loc[row, 'start_pos']] != nuc_to_watch and read[df.loc[row, 'start_pos']] != '-' \
                and read[df.loc[row, 'stop_pos']] == df.loc[row, 'ref_duplex'][1]:
                    
                    snp = read[df.loc[row, 'start_pos']]
                    
                    df.loc[row, snp] += 1
        
    return df, coverage
    

In [16]:
def create_pivot_df(df_snp):
    
    context_data = df_snp.groupby('ref_duplex').sum()
    context_data.drop(['start_pos', 'stop_pos'], inplace=True, axis=1)
    
    return context_data

    
    

## 2. main()

In [17]:
def main(in_file):
    
    nuc_to_watch = "C"
    
    snp_type = ['A', 'T', 'G']
    
    ref_seq = get_ref(in_file)
    
    duplex_posits = get_duplex_posits(ref_seq, nuc_to_watch)
    
    
    df = create_df(duplex_posits, snp_type)
    
    
    df_snp, coverage = collect_snp(in_file, df, nuc_to_watch)

    df_duplex_context = create_pivot_df(df_snp)
    
    
    return df_snp, df_duplex_context, coverage
    
    
    
    

In [18]:
df_snp, df_context, cov = main('./data/DK-A3B_S131_L001_R2_001 assembled to HBV_0.fasta')

('HBV_0 Hepatitis B virus (strain ayw) genome', 'AT-G-G-C-T-G-CTA-G-G-CTGTGCTGCCAACTGG-ATCCTGCGC----GGGAC--GTCCTTTGTTT--------ACGTC---CCGTCGGCGCTGA-ATCCT-GC--GGAC-GACCC-TT--CT-CG----------GGG-TCGCTTGG--GACTCTCTCGTCCCCT------T-CTCCGT-C---TGC-CGTTCCG-A-CC--GAC--CAC-GGG-GC-GCACCTCTCTTTACG--CGGACTCC-CC-GTCTGTGCCTTCTCAT-CTGCC-G-G--A-CC--GTG---T--G-C-A--C--T')


In [20]:
df_snp.head()

,start_pos,stop_pos,ref_duplex,A,T,G
0,7,9,CT,30.0,78.0,47.0
1,13,14,CT,24.0,55.0,68.0
2,21,22,CT,20.0,34.0,44.0
3,26,27,CT,10.0,14.0,16.0
4,29,30,CC,21.0,17.0,21.0


In [21]:
df_context

,A,T,G
ref_duplex,,,
CA,122.0,160.0,68.0
CC,674.0,965.0,261.0
CG,822.0,2042.0,507.0
CT,979.0,684.0,523.0


In [22]:
cov

8928

In [4]:
a = ["A", "T", "G", "C"]

In [5]:
a.remove("A")

In [6]:
a

['T', 'G', 'C']

In [7]:
a.remove("T")

In [8]:
a

['G', 'C']